In [1]:
import os
import pandas as pd
import geopandas as gpd

In [2]:
start_date = '2020-03-01'
end_date = '2022-03-31'

In [ ]:
policy_response_df = pd.read_csv(os.path.join("policy_response", "covid-policy-tracker-master/data/OxCGRT_latest.csv"), low_memory=False)
policy_response_df = policy_response_df[(policy_response_df['CountryName'] == "United Kingdom") & (
    policy_response_df['Jurisdiction'] == "STATE_TOTAL")]
policy_variables = ['C1_School closing', 'C2_Workplace closing', 'C3_Cancel public events',
                    'C4_Restrictions on gatherings', 'C5_Close public transport', 'C6_Stay at home requirements',
                    'C7_Restrictions on internal movement', 'C8_International travel controls',
                    'H3_Contact tracing', 'H6_Facial Coverings']
policy_response_df = policy_response_df.rename(columns={"Date": "date"})
policy_response_df = policy_response_df[['RegionName', 'date'] + policy_variables]
policy_response_df['date'] = pd.to_datetime(
    policy_response_df['date'], format="%Y%m%d")
policy_response_df = policy_response_df[(policy_response_df["date"] >= start_date)
                                        & (policy_response_df["date"] <= end_date)]
policy_response_df['date'] = policy_response_df['date'].dt.strftime('%Y-%m-%d')
policy_response_df = policy_response_df.sort_values(['RegionName', 'date'])
policy_response_df = policy_response_df.reset_index()
policy_response_df = policy_response_df.drop('index', axis=1)
policy_response_df.head()


In [ ]:
# Temporal interpolation
for region in policy_response_df['RegionName'].unique():
    for policy_variable in policy_variables:
        policy_response_df.loc[policy_response_df['RegionName']==region, policy_variable] = policy_response_df.loc[policy_response_df['RegionName']==region, policy_variable].bfill()
policy_response_df.head()

In [ ]:
ltlas_gdf = gpd.GeoDataFrame.from_file(os.path.join("gis", 'lad19.geojson'))
ltlas_gdf.head()

In [ ]:
policy_response_gdf = []
for idx, row in ltlas_gdf.iterrows():
    district_name = row['district_name']
    district_id = row['district_id']
    if district_id.startswith("E"):
        one_df = policy_response_df[policy_response_df['RegionName']=="England"].copy()
    elif district_id.startswith("N"):
        one_df = policy_response_df[policy_response_df['RegionName']=="Northern Ireland"].copy()
    elif district_id.startswith("W"):
        one_df = policy_response_df[policy_response_df['RegionName']=="Wales"].copy()
    elif district_id.startswith("S"):
        one_df = policy_response_df[policy_response_df['RegionName']=="Scotland"].copy()
    else:
        print("Unknown district prefix: %s" % district_name)
    one_df['district_name'] = district_name
    one_df['district_id'] = district_id
    policy_response_gdf.append(one_df)

policy_response_gdf = pd.concat(policy_response_gdf, axis=0).reset_index()
policy_response_gdf = policy_response_gdf[[
    'district_name', 'district_id', 'date'] + policy_variables]
updated_columns = {}
updated_policy_variables = []
for policy_variable in policy_variables:
    updated_policy_variable = "_".join(
        map(lambda x: x.lower(), policy_variable.split("_")[1].split()))
    updated_columns[policy_variable] = updated_policy_variable
    updated_policy_variables.append(updated_policy_variable)
policy_response_gdf = policy_response_gdf.rename(columns=updated_columns)
policy_response_gdf = policy_response_gdf.sort_values(['district_name', 'date'])
policy_response_gdf = policy_response_gdf.reset_index()
policy_response_gdf = policy_response_gdf.drop('index', axis=1)
policy_response_gdf.head()


In [7]:
policy_response_gdf[updated_policy_variables] = policy_response_gdf[updated_policy_variables].astype(float)
policy_response_gdf.to_csv("../policy_response.csv", float_format="%.1f", na_rep="N/A", index=False)